# **Instructions:**
* Update the paths (datasets, any existing checkpoints, etc)

In [3]:
!pip install ipywidgets==7.6.0
!pip install transformers
!pip install datasets
!pip install sklearn
!pip install to_pandas
!pip install numpy
!pip install tqdm


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a la

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [4]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
#from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import numpy as np
import random


C:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_dataset = pd.read_csv("C:/javaanalyser/VDET-for-Java/models/train.csv")
val_dataset = pd.read_csv("C:/javaanalyser/VDET-for-Java/models/val.csv")

# string to arrays
from ast import literal_eval
train_dataset['one-hot'] = train_dataset['one-hot'].apply(literal_eval)
val_dataset['one-hot'] = val_dataset['one-hot'].apply(literal_eval)

In [6]:
EPOCHS = 10
BATCH_SIZE = 12
tokenizer = transformers.AutoTokenizer.from_pretrained("CAUKiel/JavaBERT") 

class vulnerabilityClassifier(nn.Module):
    DROPOUT_PROB = 0.1 # default value
    N_CLASSES = 23 

    def __init__(self):
        super(vulnerabilityClassifier, self).__init__()
        self.model = transformers.AutoModel.from_pretrained("CAUKiel/JavaBERT", output_hidden_states=True) 
        self.dropout = nn.Dropout(self.DROPOUT_PROB) 
        self.linear1 = nn.Linear(768 * 4, 512)  # Add a hidden layer (adjust size as needed)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(512, self.N_CLASSES) # If you are using last four hidden state
        self.step_scheduler_after = "batch"


    def forward(self, ids, mask):
        """Use last four hidden states"""
        all_hidden_states = torch.stack(self.model(ids, attention_mask=mask)["hidden_states"])

        concatenate_pooling = torch.cat(
            (all_hidden_states[-1], all_hidden_states[-2], all_hidden_states[-3], all_hidden_states[-4]),-1
        )
        concatenate_pooling = concatenate_pooling[:, 0]

        output_dropout = self.dropout(concatenate_pooling)
        output1 = self.linear1(output_dropout)
        output_relu = self.relu(output1)
        output = self.linear2(output_relu)
        return output

def get_model():
  model = vulnerabilityClassifier()
  return model

In [7]:
import datetime

def format_time(elapsed):
    '''Takes a time in seconds and returns a string hh:mm:ss'''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [8]:
def tokenize_truncate(tokenizer, text_samples, max_length):
    full_input_ids = []

    # For each training example...
    for text in text_samples:
        # Tokenize the sample.
        input_ids = tokenizer.encode(text=text,              # Text to encode.
                                    add_special_tokens=True, # Do add specials.
                                    max_length=max_length,      # Do Truncate!
                                    truncation=True,         # Do Truncate!
                                    padding=False)           # DO NOT pad.
                                    
        # Add the tokenized result to our list.
        full_input_ids.append(input_ids)
        
    print('DONE. {:>10,} samples\n'.format(len(full_input_ids)))
    return full_input_ids


def build_batches(samples, batch_size):
    # List of batches that we'll construct.
    batch_ordered_text = []
    batch_ordered_labels = []

    print('Creating batches of size {:}...'.format(batch_size))

    # Loop over all of the input samples...    
    while len(samples) > 0:
        # `to_take` is our actual batch size. It will be `batch_size` until 
        # we get to the last batch, which may be smaller. 
        to_take = min(batch_size, len(samples))

        # Pick a random index in the list of remaining samples to start
        # our batch at.
        select = random.randint(0, len(samples) - to_take)

        # Select a contiguous batch of samples starting at `select`.
        batch = samples[select:(select + to_take)]

        #print("Batch length:", len(batch))

        # Each sample is a tuple--split them apart to create a separate list of 
        # sequences and a list of labels for this batch.
        batch_ordered_text.append([s[0] for s in batch])
        batch_ordered_labels.append([s[1] for s in batch])

        # Remove these samples from the list.
        del samples[select:select + to_take]

    print('\t  DONE - Selected {:,} batches.\n'.format(len(batch_ordered_text)))
    return batch_ordered_text, batch_ordered_labels


def add_padding_per_batch(tokenizer, batch_ordered_text, batch_ordered_labels):
    print('Padding out sequences within each batch...')

    final_input_ids = []
    final_attention_masks = []
    final_labels = []

    # For each batch...
    for (batch_inputs, batch_labels) in zip(batch_ordered_text, batch_ordered_labels):

        # New version of the batch, this time with padded sequences and now with
        # attention masks defined.
        batch_padded_inputs = []
        batch_attn_masks = []
        
        # First, find the longest sample in the batch. 
        # Note that the sequences do currently include the special tokens!
        max_size = max([len(sen) for sen in batch_inputs])

        # For each input in this batch...
        for sen in batch_inputs:
            
            # How many pad tokens do we need to add?
            num_pads = max_size - len(sen)

            # Add `num_pads` padding tokens to the end of the sequence.
            padded_input = sen + [tokenizer.pad_token_id]*num_pads

            # Define the attention mask--it's just a `1` for every real token
            # and a `0` for every padding token.
            attn_mask = [1] * len(sen) + [0] * num_pads

            # Add the padded results to the batch.
            batch_padded_inputs.append(padded_input)
            batch_attn_masks.append(attn_mask)

        # Our batch has been padded, so we need to save this updated batch.
        # We also need the inputs to be PyTorch tensors, so we'll do that here.
        # Todo - Michael's code specified "dtype=torch.long"
        final_input_ids.append(torch.tensor(batch_padded_inputs))
        final_attention_masks.append(torch.tensor(batch_attn_masks))
        final_labels.append(torch.tensor(np.array(batch_labels))) # if there's problems, remove np.array()

    print('\t DONE. Returning final smart-batched data.')
    # Return the smart-batched dataset!
    return (final_input_ids, final_attention_masks, final_labels)


def smart_batching(tokenizer, max_length, text_samples, labels, batch_size):
    # Tokenize and truncate text_samples; no padding
    full_input_ids = tokenize_truncate(tokenizer, text_samples, max_length)

    # Sort the two lists together by the length of the input sequence.
    samples = sorted(zip(full_input_ids, labels), key=lambda x: len(x[0]))

    # Build batches of contiguous data, starting at random points in samples
    batch_size = batch_size
    batch_ordered_text, batch_ordered_labels = build_batches(samples, batch_size)
   
    # Add padding accordingly to batch size
    final_input_ids, final_attention_masks, final_labels = add_padding_per_batch(tokenizer, batch_ordered_text, batch_ordered_labels)

    return final_input_ids, final_attention_masks, final_labels



In [9]:
train_input_ids, train_attn_masks, train_labels = smart_batching(tokenizer, 512, train_dataset['Code'], train_dataset['one-hot'], BATCH_SIZE) 
# test_input_ids, test_attn_masks, test_labels = smart_batching(tokenizer, 512, test_dataset['Code'], test_dataset['one-hot'], BATCH_SIZE)
val_input_ids, val_attn_masks, val_labels = smart_batching(tokenizer, 512, val_dataset['Code'], val_dataset['one-hot'], BATCH_SIZE)


DONE.     92,484 samples

Creating batches of size 12...
	  DONE - Selected 7,707 batches.

Padding out sequences within each batch...
	 DONE. Returning final smart-batched data.
DONE.     11,589 samples

Creating batches of size 12...
	  DONE - Selected 966 batches.

Padding out sequences within each batch...
	 DONE. Returning final smart-batched data.


In [10]:
def get_optimizer(model):
    opt = torch.optim.AdamW(model.parameters(),
                  lr = 5e-5,                   
                  eps = 1e-8                    
                )

    return opt

def get_scheduler(optimizer, num_train_steps):
    sch = transformers.get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return sch

In [11]:
def loss_fn(outputs, labels):
    if labels is None:
        return None
    # num_output_elements = len(outputs)
    # num_target_elements = len(labels)

    # print("Number of elements in lossfn outputs:", num_output_elements)
    # print("Number of elements in lossfn labels:", num_target_elements)
    return nn.BCEWithLogitsLoss()(outputs, labels.float())

In [12]:
from sklearn.metrics import accuracy_score

def getAccuracy(preds, labels):
    prob_preds = torch.stack(preds)
    prob_preds = prob_preds.cpu().detach().numpy()
    flabels = torch.stack(labels)
    flabels = flabels.cpu().detach().numpy()

    label_predictions = np.zeros((len(preds), 23))
    label_predictions = prob_preds >= 0.5 
    label_predictions = label_predictions.astype(int)

    # accuracy_score from sklearn calculates subset accuracy 
    return accuracy_score(flabels, label_predictions)

In [13]:
import time

def train_fn(train_input_ids, train_attn_masks, train_labels, model, optimizer,scheduler):
    update_interval = 500
    t0 = time.time()
    train_loss = 0.0
    model.train()
    final_targets = []
    final_outputs = []
    
    for step in range(0, len(train_input_ids)): 
        if step % update_interval == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            
            steps_per_sec = (time.time() - t0) / step
            remaining_sec = steps_per_sec * (len(train_input_ids) - step)
            remaining = format_time(remaining_sec)
            print('  Batch {:>7,}  of  {:>7,}.    Elapsed: {:}.  Remaining: {:}'.format(step, len(train_input_ids), elapsed, remaining))

        if torch.cuda.is_available():
            ids = train_input_ids[step].to('cuda', dtype=torch.long)
            mask = train_attn_masks[step].to('cuda', dtype=torch.long)
            targets = train_labels[step].to('cuda', dtype=torch.float)
        else:
            ids = train_input_ids[step]
            mask = train_attn_masks[step]
            targets = train_labels[step]

        optimizer.zero_grad()
        outputs = model(ids=ids, mask=mask)
        loss = loss_fn(outputs, targets)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        scheduler.step()
        final_targets.extend(targets)
        final_outputs.extend(torch.sigmoid(outputs))
        
        
    return train_loss, final_outputs, final_targets
    

In [14]:
import time
def eval_fn(test_input_ids, test_attn_masks, test_labels, model):
    print('\nStarting evaluation... ')
    
    update_interval = 100
    t0 = time.time()

    eval_loss = 0.0

    model.eval()

    final_targets = []
    final_outputs = []

    with torch.no_grad():
       for step in range(0, len(test_input_ids)): 
          if step % update_interval == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Calculate the time remaining based on our progress.
              steps_per_sec = (time.time() - t0) / step
              remaining_sec = steps_per_sec * (len(test_input_ids) - step)
              remaining = format_time(remaining_sec)
              # Report progress.
              print('  Batch {:>7,}  of  {:>7,}.    Elapsed: {:}.  Remaining: {:}'.format(step, len(test_input_ids), elapsed, remaining))

          if torch.cuda.is_available():
              ids = test_input_ids[step].to('cuda', dtype = torch.long)
              mask = test_attn_masks[step].to('cuda', dtype = torch.long)
              targets = test_labels[step].to('cuda', dtype = torch.float)
          else:
              # Train on CPU
              ids = test_input_ids[step]
              mask = test_attn_masks[step]
              targets = test_labels[step]
           
          # ids = test_input_ids[step].to('cuda', dtype = torch.long)
          # mask = test_attn_masks[step].to('cuda', dtype = torch.long)
          # targets = test_labels[step].to('cuda', dtype = torch.float)

          outputs = model(ids=ids, mask=mask)

          loss = loss_fn(outputs, targets)

          eval_loss += loss.item()
          final_targets.extend(targets)
          final_outputs.extend(torch.sigmoid(outputs))
 
    return eval_loss, final_outputs, final_targets

In [16]:
def save_checkpoint(epoch, optimizer, scheduler, model, train_loss, test_loss):
  torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'test_loss': test_loss
            }, "C:/javaanalyser/VDET-for-Java/models/model.pt")
  
  print("Saved hs_checkpoint_" + str(epoch) + ".pt")

In [15]:
from tqdm import tqdm

n_train_steps = len(train_input_ids) * EPOCHS

model = get_model()
if torch.cuda.is_available():
    model.to('cuda')
else:
    print("CUDA not available. Training on CPU.")

optimizer = get_optimizer(model)
scheduler = get_scheduler(optimizer, n_train_steps)

# Load checkpoint from a previous model
# checkpoint = torch.load('C:/javaanalyser/VDET-for-Java/plugin-backend/assets/model.bin')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
# best_eval_loss = checkpoint['test_loss']
# epoch = checkpoint['epoch'] + 1
epoch = 0

Some weights of BertModel were not initialized from the model checkpoint at CAUKiel/JavaBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


CUDA not available. Training on CPU.


In [16]:
while epoch < EPOCHS:
    print("\t\t epoch: ", epoch)

    if epoch > 0: # This is not cross-validation! 
      train_input_ids, train_attn_masks, train_labels = smart_batching(tokenizer, 512, train_dataset['Code'], train_dataset['one-hot'], 12) 
      val_input_ids, val_attn_masks, val_labels = smart_batching(tokenizer, 512, val_dataset['Code'], val_dataset['one-hot'], 12)

    train_loss, train_preds, train_true_labels = train_fn(train_input_ids, train_attn_masks, train_labels, model, optimizer, scheduler)
    eval_loss, eval_preds, eval_true_labels = eval_fn(val_input_ids, val_attn_masks, val_labels, model)

    avg_train_loss, avg_val_loss = train_loss / len(train_input_ids), eval_loss / len(val_input_ids)
    train_acc = getAccuracy(train_preds, train_true_labels) 
    eval_acc = getAccuracy(eval_preds, eval_true_labels)

    train_info = "Avg Train loss (loss/batch): " + str(avg_train_loss) +  "\t Train accuracy: " + str(train_acc) + "\n"
    val_info = "Avg Valid loss (loss/batch): " + str(avg_val_loss) + "\t Validation accuracy: " + str(eval_acc) + "\n\n"

    f = open("C:/javaanalyser/VDET-for-Java/models/loss.txt", "a")
    f.write(train_info)
    f.write(val_info)
    print(train_info)
    print(val_info)
    f.close()
    optimizer.step()
    scheduler.step()
    save_checkpoint(epoch, optimizer, scheduler, model, train_loss, eval_loss)
    epoch = epoch + 1



		 epoch:  0
Starting training... 
  Batch     500  of    7,707.    Elapsed: 0:34:37.  Remaining: 8:18:55
  Batch   1,000  of    7,707.    Elapsed: 1:08:35.  Remaining: 7:40:02
  Batch   1,500  of    7,707.    Elapsed: 1:46:32.  Remaining: 7:20:52
  Batch   2,000  of    7,707.    Elapsed: 2:36:31.  Remaining: 7:26:37
  Batch   2,500  of    7,707.    Elapsed: 3:13:45.  Remaining: 6:43:32
  Batch   3,000  of    7,707.    Elapsed: 3:49:42.  Remaining: 6:00:24
  Batch   3,500  of    7,707.    Elapsed: 4:23:02.  Remaining: 5:16:10
  Batch   4,000  of    7,707.    Elapsed: 4:58:31.  Remaining: 4:36:39
  Batch   4,500  of    7,707.    Elapsed: 5:34:33.  Remaining: 3:58:26
  Batch   5,000  of    7,707.    Elapsed: 6:07:49.  Remaining: 3:19:08
  Batch   5,500  of    7,707.    Elapsed: 6:44:56.  Remaining: 2:42:29
  Batch   6,000  of    7,707.    Elapsed: 7:25:44.  Remaining: 2:06:49
  Batch   6,500  of    7,707.    Elapsed: 8:18:13.  Remaining: 1:32:31
  Batch   7,000  of    7,707.    Elapsed: 

KeyboardInterrupt: 

From this cell onwards, we just test the final checkpoint:

In [17]:
from tqdm import tqdm

model = get_model()
if torch.cuda.is_available():
    model.to('cuda')

# Load best model iteration for testing
checkpoint = torch.load('C:/javaanalyser/VDET-for-Java/plugin-backend/model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
epoch = checkpoint['epoch']
best_eval_loss = checkpoint['test_loss']


Some weights of BertModel were not initialized from the model checkpoint at CAUKiel/JavaBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'optimizer' is not defined

In [18]:
from ast import literal_eval

test_dataset = pd.read_csv("C:/javaanalyser/VDET-for-Java/models/test.csv")
test_dataset['one-hot'] = test_dataset['one-hot'].apply(literal_eval)

test_input_ids, test_attn_masks, test_labels = smart_batching(tokenizer, 512, test_dataset['Code'], test_dataset['one-hot'], BATCH_SIZE)

test_loss, test_preds, test_true_labels = eval_fn(test_input_ids, test_attn_masks, test_labels, model)

DONE.     11,527 samples

Creating batches of size 12...
	  DONE - Selected 961 batches.

Padding out sequences within each batch...
	 DONE. Returning final smart-batched data.

Starting evaluation... 


KeyboardInterrupt: 

In [55]:
test_accuracy = getAccuracy(test_preds, test_true_labels)

#subset accuracy
print("Testing accuracy:" , test_accuracy) 

Testing accuracy: 0.8629305109742345


In [20]:
from sklearn.metrics import multilabel_confusion_matrix

prob_preds = torch.stack(test_preds)
prob_preds = prob_preds.cpu().detach().numpy()

label_predictions = (prob_preds > 0.5).astype(int) 

flabels = torch.stack(test_true_labels)
flabels = flabels.cpu().detach().numpy()


# Build confusion matrix
cf_matrix = multilabel_confusion_matrix(flabels, label_predictions) #use this to get FP, FN, TP, TN to calculate accuracy

cf_matrix

array([[[10770,    39],
        [    9,   709]],

       [[10271,     0],
        [  103,  1153]],

       [[11138,     0],
        [   25,   364]],

       [[11357,     0],
        [   13,   157]],

       [[ 9308,     4],
        [  163,  2052]],

       [[ 9741,    66],
        [   94,  1626]],

       [[11109,     1],
        [   34,   383]],

       [[11459,     0],
        [   15,    53]],

       [[11353,     0],
        [    2,   172]],

       [[11423,     0],
        [   12,    92]],

       [[10609,     0],
        [   40,   878]],

       [[10827,     0],
        [    0,   700]],

       [[11367,     0],
        [    2,   158]],

       [[11306,     0],
        [   20,   201]],

       [[11294,     0],
        [   15,   218]],

       [[11373,     0],
        [    1,   153]],

       [[11377,     0],
        [    9,   141]],

       [[11022,     6],
        [   28,   471]],

       [[11285,     0],
        [   21,   221]],

       [[10521,    32],
        [   37,   937]],



In [21]:
from sklearn.metrics import classification_report
cr = classification_report(flabels, label_predictions)
print(cr)

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       718
           1       1.00      0.92      0.96      1256
           2       1.00      0.94      0.97       389
           3       1.00      0.92      0.96       170
           4       1.00      0.93      0.96      2215
           5       0.96      0.95      0.95      1720
           6       1.00      0.92      0.96       417
           7       1.00      0.78      0.88        68
           8       1.00      0.99      0.99       174
           9       1.00      0.88      0.94       104
          10       1.00      0.96      0.98       918
          11       1.00      1.00      1.00       700
          12       1.00      0.99      0.99       160
          13       1.00      0.91      0.95       221
          14       1.00      0.94      0.97       233
          15       1.00      0.99      1.00       154
          16       1.00      0.94      0.97       150
          17       0.99    

In [20]:
import requests

In [21]:
# url = "http://ec2-18-191-178-89.us-east-2.compute.amazonaws.com:5000/predict/section"

In [52]:
test_code = """
public void method() throws Throwable {
  data = "";
  {
    ServerSocket listener = null;
    Socket socket = null;
    BufferedReader readerBuffered = null;
    InputStreamReader readerInputStream = null;
    try {
      listener = new ServerSocket(39543);
      socket = listener.accept();
      readerInputStream = new InputStreamReader(socket.getInputStream(), "UTF-8");
      readerBuffered = new BufferedReader(readerInputStream);
      data = readerBuffered.readLine();
    } catch (IOException exceptIO) {
      IO.logger.log(Level.WARNING, "Error with stream reading", exceptIO);
    } finally {
      try {
        if (readerBuffered != null) {
          readerBuffered.close();
        }
      } catch (IOException e) {
        e.printStackTrace();
      }
      try {
        if (socket != null)
          socket.close();
      } catch (IOException e) {
        e.printStackTrace();
      }
      try {
        if (listener != null)
          listener.close();
      } catch (IOException e) {
        e.printStackTrace();
      }
    }
  }
}
"""

# Since you want to analyze all lines, set test_lines to an empty list
test_lines = []  # Empty list to include all lines

In [53]:

# Assuming your server runs on http://localhost:5000 and endpoint is /predict/section
url = "http://localhost:9696/predict/section"

data = {"code": test_code, "lines": test_lines}

response = requests.post(url, json=data)

In [54]:
if response.status_code == 200:
  # Request successful
  data = response.json()
  print(f"Predicted labels for all lines:")
  for result in data:
      print(f"\tLines: {result['range']}")
      print(f"\tLabels: {result['labels']}")
else:
  print(f"Error: {response.status_code}")

Predicted labels for all lines:
	Lines: 
	Labels: [['113', '0.5179373'], ['197', '0.55499643'], ['319', '0.53877366'], ['36', '0.53712976'], ['369', '0.5516819'], ['606', '0.606603'], ['690', '0.501983'], ['80', '0.5468344'], ['89', '0.5148977'], ['90', '0.5462103'], ['False', '0.5237346']]


In [55]:
test_code2 = """
public void method(String data) throws Throwable {    (new CWE89_SQL_Injection__PropertiesFile_prepareStatement_53c()).goodG2BSink(data);}
"""

# Since you want to analyze all lines, set test_lines to an empty list
test_lines2 = []  # Empty list to include all lines

In [56]:
data2 = {"code": test_code2, "lines": test_lines2}

response = requests.post(url, json=data2)

In [57]:
if response.status_code == 200:
  # Request successful
  data = response.json()
  print(f"Predicted labels for all lines:")
  for result in data:
      print(f"\tLines: {result['range']}")
      print(f"\tLabels: {result['labels']}")
else:
  print(f"Error: {response.status_code}")

Predicted labels for all lines:
	Lines: 
	Labels: [['129', '0.5297037'], ['15', '0.5104554'], ['190', '0.5280324'], ['197', '0.59009415'], ['23', '0.5092397'], ['319', '0.5115453'], ['369', '0.51730555'], ['470', '0.50223327'], ['606', '0.5060164'], ['643', '0.51146704'], ['78', '0.51714903'], ['80', '0.67145765'], ['90', '0.50398505'], ['True', '0.5112574']]


In [49]:
def test_predict_non_vulnerable_code():
  # Implement logic to mock data with a known non-vulnerable code snippet
  data = {"code": "safe_code"}
  response = request.post(url, json=data)
  assert "vulnerable_label" not in response

In [47]:
def test_predict_empty_code():
  data = {"code": ""}
  with pytest.raises(Exception):
    predict(data)

In [51]:
# Mock data for section prediction
def test_predict_section():
  data = {"code": "code with sections", "lines": [1, 3]}
  response = request.post(url, json=data)
  assert len(response) == 2  # One result for each line

# Mock data for file prediction
def test_predict_file():
  data = [{"code": "code1", "name": "file1"}, {"code": "code2", "name": "file2"}]
  response = request.post(url, json=data)
  assert len(response) == 2  # One result for each file